<a href="https://colab.research.google.com/github/suhaasvarma-05/Detection-of-Brain-Tumors-Using-YOLOv11-with-XAI-/blob/main/brain_tumour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 53.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [ ]:
from roboflow import Roboflow

In [ ]:
rf=Roboflow(api_key="CE4lpbKi4sP2Hk9PrrEB")
project=rf.workspace("braindata").project("brain-tumor-detection-aaerl")
version=project.version(1)
dataset=version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Brain-Tumor-Detection-1 in yolov11:: 100%|██████████| 9128/9128 [00:01<00:00, 7648.91it/s]


In [ ]:
!pip install ultralytics opencv-python pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.2 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO



model = YOLO("yolo11n.pt")



# Train the model

train_results = model.train(

    data="/content/Brain-Tumor-Detection-1/data.yaml",  # path to dataset YAML

    epochs=50,  # number of training epochs

    imgsz=640,  # training image size

    device=0,  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu

)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


In [ ]:
from ultralytics import YOLO



# Load a model

model = YOLO("/content/runs/detect/train/weights/best.pt")



# Perform object detection on an image

results = model("/content/Brain-Tumor-Detection-1/test/images/Te-me_0130_jpg.rf.0f612b364bfe9f63fcfdedfcb30cf90c.jpg", save=True)

results[0].show()

In [ ]:
from ultralytics import YOLO

from ultralytics import SAM



# Load the YOLO model

yolo_model = YOLO("/content/runs/detect/train/weights/best.pt")  # pretrained YOLO model



# Run batched inference on a list of images

results = yolo_model("/content/Brain-Tumor-Detection-1/test/images/Te-me_0130_jpg.rf.0f612b364bfe9f63fcfdedfcb30cf90c.jpg")
sam_model = SAM("sam2_b.pt")



for result in results:

     class_ids = result.boxes.cls.int().tolist()  # noqa

     if len(class_ids):

         boxes = result.boxes.xyxy  # Boxes object for bbox outputs

         sam_results = sam_model(result.orig_img, bboxes=boxes, verbose=False, save=True, device=0)

In [ ]:
# Extract and display accuracy metrics
# mAP50 is a common metric for object detection accuracy
print("Accuracy (mAP50):", train_results.results_dict['metrics/mAP50(B)'])

In [ ]:
from IPython.display import Image, display
display(Image('/content/runs/segment/predict/image0.jpg'))

In [ ]:
# Show Confusion Matrix
from IPython.display import Image, display
display(Image('/content/runs/detect/train/confusion_matrix_normalized.png'))

In [ ]:
# Show Confusion Matrix
from IPython.display import Image, display
display(Image('/content/runs/detect/predict/Te-me_0130_jpg.rf.0f612b364bfe9f63fcfdedfcb30cf90c.jpg'))

In [ ]:
# Install required packages (no pytorch-grad-cam needed)
!pip install ultralytics opencv-python pandas matplotlib seaborn

import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from PIL import Image
import torch.nn.functional as F

# ============================================
# 1. Improved YOLO Grad-CAM Implementation
# ============================================

class YOLOGradCAM:
    """Grad-CAM implementation for YOLO models"""

    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def generate_gradcam(self, image_path):
        """Generate Grad-CAM heatmap for an image"""

        # Read and preprocess image
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        original_shape = img_rgb.shape[:2]

        # Get prediction first
        results = self.model(image_path, verbose=False)

        # Prepare image for processing
        img_resized = cv2.resize(img_rgb, (640, 640))
        img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).unsqueeze(0).float() / 255.0
        img_tensor = img_tensor.to(self.device)
        img_tensor.requires_grad = True

        # Storage for features and gradients
        features = []

        def forward_hook(module, input, output):
            features.append(output.clone().detach().requires_grad_(True))

        # Register hook on a middle layer
        target_layer = self.model.model.model[-4]  # Use layer before head
        handle = target_layer.register_forward_hook(forward_hook)

        try:
            # Forward pass
            with torch.set_grad_enabled(True):
                output = self.model.model(img_tensor)

            # Get the feature map
            if len(features) > 0:
                feature_map = features[0]

                # Create a simple target score (sum of all outputs)
                if isinstance(output, (list, tuple)):
                    # For YOLO v8/v11 format
                    target = output[0].sum()
                else:
                    target = output.sum()

                # Compute gradients
                target.backward(retain_graph=True)

                # Get gradients
                if feature_map.grad is not None:
                    gradients = feature_map.grad

                    # Global average pooling on gradients
                    weights = torch.mean(gradients, dim=[2, 3], keepdim=True)

                    # Weighted combination of feature maps
                    cam = torch.sum(weights * feature_map, dim=1, keepdim=True)
                    cam = F.relu(cam)

                    # Normalize
                    cam = cam.squeeze().cpu().detach().numpy()
                    cam = cam - cam.min()
                    if cam.max() > 0:
                        cam = cam / cam.max()

                    # Resize to original image size
                    cam = cv2.resize(cam, (original_shape[1], original_shape[0]))
                else:
                    # Fallback: use feature map activations
                    cam = torch.mean(feature_map, dim=1).squeeze().cpu().detach().numpy()
                    cam = np.maximum(cam, 0)
                    if cam.max() > 0:
                        cam = cam / cam.max()
                    cam = cv2.resize(cam, (original_shape[1], original_shape[0]))
            else:
                # Create empty heatmap
                cam = np.zeros(original_shape)

        except Exception as e:
            print(f"Grad-CAM error: {e}")
            # Fallback: create heatmap based on detection boxes
            cam = np.zeros(original_shape)
            if len(results[0].boxes) > 0:
                for box, conf in zip(results[0].boxes.xyxy, results[0].boxes.conf):
                    x1, y1, x2, y2 = map(int, box.tolist())
                    cam[y1:y2, x1:x2] = conf.item()
                if cam.max() > 0:
                    cam = cam / cam.max()

        finally:
            handle.remove()

        return cam, img_rgb, results

# ============================================
# 2. Saliency Map Implementation
# ============================================

def generate_saliency_map(model, image_path):
    """Generate saliency map showing pixel importance"""

    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    original_shape = img_rgb.shape[:2]

    try:
        img_resized = cv2.resize(img_rgb, (640, 640))
        img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).unsqueeze(0).float() / 255.0
        img_tensor = img_tensor.to(model.device)
        img_tensor.requires_grad = True

        # Forward pass
        output = model.model(img_tensor)

        # Compute score
        if isinstance(output, (list, tuple)):
            score = output[0].sum()
        else:
            score = output.sum()

        # Backward pass
        score.backward()

        # Get gradients
        saliency = img_tensor.grad.data.abs()
        saliency = saliency.squeeze(0).permute(1, 2, 0).cpu().numpy()
        saliency = np.max(saliency, axis=2)

        # Normalize
        if saliency.max() > 0:
            saliency = saliency / saliency.max()

        # Resize to original
        saliency = cv2.resize(saliency, (original_shape[1], original_shape[0]))

    except Exception as e:
        print(f"Saliency map error: {e}")
        saliency = np.zeros(original_shape)

    return saliency, img_rgb

# ============================================
# 3. Attention Visualization
# ============================================

def visualize_attention_regions(results, image_path):
    """Visualize detected regions with confidence scores"""

    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Create attention map
    attention_map = np.zeros((img.shape[0], img.shape[1]), dtype=np.float32)

    if len(results[0].boxes) > 0:
        for box, conf in zip(results[0].boxes.xyxy, results[0].boxes.conf):
            x1, y1, x2, y2 = map(int, box.tolist())
            # Ensure coordinates are within image bounds
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(img.shape[1], x2), min(img.shape[0], y2)

            # Add weighted attention
            attention_map[y1:y2, x1:x2] += conf.item()

    # Normalize
    if attention_map.max() > 0:
        attention_map = attention_map / attention_map.max()

    return attention_map, img_rgb

# ============================================
# 4. Feature Visualization
# ============================================

def extract_and_visualize_features(model, image_path):
    """Extract and visualize intermediate feature maps"""

    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (640, 640))
    img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    img_tensor = img_tensor.to(model.device)

    features = []

    def hook(module, input, output):
        features.append(output.clone().detach())

    # Hook into multiple layers
    layers_to_hook = [-6, -4, -2]
    handles = []

    for layer_idx in layers_to_hook:
        try:
            target_layer = model.model.model[layer_idx]
            handle = target_layer.register_forward_hook(hook)
            handles.append(handle)
        except:
            pass

    # Forward pass
    with torch.no_grad():
        _ = model.model(img_tensor)

    # Remove hooks
    for handle in handles:
        handle.remove()

    return features, img_rgb

# ============================================
# 5. Comprehensive Visualization
# ============================================

def comprehensive_xai_visualization(model_path, image_path, output_path='xai_analysis.png'):
    """Create comprehensive XAI visualization"""

    print("Loading model and generating XAI visualizations...")

    # Load model
    model = YOLO(model_path)

    # Original prediction
    print("Running inference...")
    results = model(image_path, verbose=False)

    # Initialize Grad-CAM
    print("Generating Grad-CAM...")
    gradcam = YOLOGradCAM(model_path)
    heatmap, img_rgb, _ = gradcam.generate_gradcam(image_path)

    print("Generating Saliency Map...")
    saliency, _ = generate_saliency_map(model, image_path)

    print("Generating Attention Map...")
    attention, _ = visualize_attention_regions(results, image_path)

    print("Extracting Features...")
    features, _ = extract_and_visualize_features(model, image_path)

    # Create visualization
    print("Creating visualization...")
    fig = plt.figure(figsize=(20, 14))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

    # Row 1: Original, Prediction, Grad-CAM
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.imshow(img_rgb)
    ax1.set_title('Original Image', fontsize=16, fontweight='bold', pad=10)
    ax1.axis('off')

    ax2 = fig.add_subplot(gs[0, 1])
    pred_img = results[0].plot()
    ax2.imshow(cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB))
    ax2.set_title('Model Prediction', fontsize=16, fontweight='bold', pad=10)
    ax2.axis('off')

    ax3 = fig.add_subplot(gs[0, 2])
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    heatmap_colored = cv2.cvtColor(heatmap_colored, cv2.COLOR_BGR2RGB)
    superimposed = cv2.addWeighted(img_rgb, 0.5, heatmap_colored, 0.5, 0)
    ax3.imshow(superimposed)
    ax3.set_title('Grad-CAM Overlay', fontsize=16, fontweight='bold', pad=10)
    ax3.axis('off')

    # Row 2: Saliency, Attention, Pure Heatmap
    ax4 = fig.add_subplot(gs[1, 0])
    im4 = ax4.imshow(saliency, cmap='hot')
    ax4.set_title('Saliency Map (Pixel Importance)', fontsize=16, fontweight='bold', pad=10)
    ax4.axis('off')
    plt.colorbar(im4, ax=ax4, fraction=0.046)

    ax5 = fig.add_subplot(gs[1, 1])
    im5 = ax5.imshow(attention, cmap='viridis')
    ax5.set_title('Attention Regions', fontsize=16, fontweight='bold', pad=10)
    ax5.axis('off')
    plt.colorbar(im5, ax=ax5, fraction=0.046)

    ax6 = fig.add_subplot(gs[1, 2])
    im6 = ax6.imshow(heatmap, cmap='jet')
    ax6.set_title('Activation Heatmap', fontsize=16, fontweight='bold', pad=10)
    ax6.axis('off')
    plt.colorbar(im6, ax=ax6, fraction=0.046)

    # Row 3: Feature maps from different layers
    if features and len(features) > 0:
        for idx, (feature, ax_idx) in enumerate(zip(features[:3], [7, 8, 9])):
            ax = fig.add_subplot(gs[2, ax_idx-7])

            # Average across channels for visualization
            feat_vis = torch.mean(feature[0], dim=0).cpu().numpy()
            feat_vis = (feat_vis - feat_vis.min()) / (feat_vis.max() - feat_vis.min() + 1e-8)

            im = ax.imshow(feat_vis, cmap='viridis')
            ax.set_title(f'Feature Map - Layer {idx+1}', fontsize=14, fontweight='bold', pad=10)
            ax.axis('off')
            plt.colorbar(im, ax=ax, fraction=0.046)

    # Add detection details
    detection_text = "🔍 Detection Results:\n" + "="*50 + "\n"
    if len(results[0].boxes) > 0:
        for i, (box, conf, cls) in enumerate(zip(results[0].boxes.xyxy,
                                                  results[0].boxes.conf,
                                                  results[0].boxes.cls)):
            class_name = results[0].names[int(cls.item())]
            detection_text += f"  {i+1}. {class_name.upper()}: {conf.item()*100:.2f}% confidence\n"
    else:
        detection_text += "  No tumors detected\n"

    detection_text += "="*50 + "\n"
    detection_text += "📊 XAI Methods:\n"
    detection_text += "  • Grad-CAM: Neural activation regions\n"
    detection_text += "  • Saliency: Pixel-level importance\n"
    detection_text += "  • Attention: Confidence-weighted focus\n"

    plt.figtext(0.5, 0.01, detection_text, ha='center', fontsize=11,
               family='monospace',
               bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))

    plt.suptitle('Brain Tumor Detection - Explainable AI Analysis',
                fontsize=20, fontweight='bold', y=0.98)

    plt.savefig(output_path, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"✅ XAI visualization saved to: {output_path}")
    plt.show()

# ============================================
# 6. Generate XAI Report
# ============================================

def generate_xai_report(model_path, image_path):
    """Generate detailed XAI report"""

    model = YOLO(model_path)
    results = model(image_path, verbose=False)

    print("\n" + "="*70)
    print("🧠 BRAIN TUMOR DETECTION - EXPLAINABLE AI ANALYSIS REPORT")
    print("="*70)
    print(f"\n📁 Image: {image_path.split('/')[-1]}")
    print(f"🤖 Model: {model_path.split('/')[-1]}")
    print("\n" + "-"*70)

    if len(results[0].boxes) > 0:
        print("\n🎯 DETECTIONS:")
        total_confidence = 0
        for i, (box, conf, cls) in enumerate(zip(results[0].boxes.xyxy,
                                                  results[0].boxes.conf,
                                                  results[0].boxes.cls)):
            class_name = results[0].names[int(cls.item())]
            x1, y1, x2, y2 = box.tolist()
            area = (x2-x1)*(y2-y1)
            total_confidence += conf.item()

            print(f"\n  Detection #{i+1}:")
            print(f"    ├─ Class: {class_name.upper()}")
            print(f"    ├─ Confidence: {conf.item():.4f} ({conf.item()*100:.2f}%)")
            print(f"    ├─ Location: ({x1:.0f}, {y1:.0f}) → ({x2:.0f}, {y2:.0f})")
            print(f"    └─ Area: {area:.0f} pixels² ({(area/(results[0].orig_shape[0]*results[0].orig_shape[1]))*100:.2f}% of image)")

        avg_conf = total_confidence / len(results[0].boxes)
        print(f"\n  📊 Average Confidence: {avg_conf*100:.2f}%")
    else:
        print("\n❌ No tumors detected in the image.")

    print("\n" + "-"*70)
    print("\n💡 EXPLAINABILITY METHODS:")
    print("  1. Grad-CAM (Gradient-weighted Class Activation Mapping)")
    print("     → Highlights which regions activated the neural network")
    print("     → Shows spatial importance for tumor detection")
    print("\n  2. Saliency Map")
    print("     → Displays pixel-level importance using gradients")
    print("     → Indicates which pixels most influence the prediction")
    print("\n  3. Attention Visualization")
    print("     → Shows model's focus areas weighted by confidence")
    print("     → Combines detection locations with certainty scores")
    print("\n  4. Feature Map Analysis")
    print("     → Reveals intermediate layer representations")
    print("     → Shows patterns and features learned by the network")
    print("\n" + "="*70 + "\n")

# ============================================
# 7. Main Execution
# ============================================

# Use your trained model
MODEL_PATH = "/content/runs/detect/train/weights/best.pt"
IMAGE_PATH = "/content/Brain-Tumor-Detection-1/test/images/Te-me_0159_jpg.rf.7afad29587df72dcfba2e189a75656fa.jpg"

# Generate comprehensive XAI visualization
comprehensive_xai_visualization(MODEL_PATH, IMAGE_PATH, 'brain_tumor_xai_analysis.png')

# Display the result
from IPython.display import Image, display
display(Image('brain_tumor_xai_analysis.png'))

# Generate detailed report
generate_xai_report(MODEL_PATH, IMAGE_PATH)

In [ ]:
# Install required packages
!pip install ultralytics opencv-python pandas matplotlib seaborn numpy scipy

import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO
from scipy.spatial import ConvexHull
from pathlib import Path
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import json

# ============================================
# 1. Volumetric Analysis Core Functions
# ============================================

class BrainTumorVolumetricAnalyzer:
    """Comprehensive volumetric analysis for brain tumor detection"""

    def __init__(self, model_path, pixel_spacing=1.0, slice_thickness=1.0):
        """
        Initialize analyzer

        Args:
            model_path: Path to YOLO model
            pixel_spacing: Physical size of pixel in mm (default: 1.0 mm)
            slice_thickness: Thickness between slices in mm (default: 1.0 mm)
        """
        self.model = YOLO(model_path)
        self.pixel_spacing = pixel_spacing  # mm per pixel
        self.slice_thickness = slice_thickness  # mm between slices

    def analyze_single_image(self, image_path):
        """Perform volumetric analysis on a single image"""

        # Run detection
        results = self.model(image_path, verbose=False)

        # Read image
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        analyses = []

        if len(results[0].boxes) > 0:
            for i, (box, conf, cls) in enumerate(zip(results[0].boxes.xyxy,
                                                      results[0].boxes.conf,
                                                      results[0].boxes.cls)):

                x1, y1, x2, y2 = map(int, box.tolist())
                class_name = results[0].names[int(cls.item())]

                # Extract tumor region
                tumor_region = img_rgb[y1:y2, x1:x2]

                # Perform measurements
                analysis = self._analyze_tumor_region(
                    tumor_region,
                    (x1, y1, x2, y2),
                    class_name,
                    conf.item(),
                    img_rgb.shape
                )

                analyses.append(analysis)

        return analyses, results, img_rgb

    def _analyze_tumor_region(self, region, bbox, class_name, confidence, img_shape):
        """Analyze a single tumor region"""

        x1, y1, x2, y2 = bbox

        # 2D measurements
        width_pixels = x2 - x1
        height_pixels = y2 - y1
        area_pixels = width_pixels * height_pixels

        # Convert to physical measurements (mm)
        width_mm = width_pixels * self.pixel_spacing
        height_mm = height_pixels * self.pixel_spacing
        area_mm2 = area_pixels * (self.pixel_spacing ** 2)

        # Estimate 3D volume using different methods

        # Method 1: Ellipsoid approximation (most common for tumors)
        # V = (4/3) * π * a * b * c, where c is estimated from average of a and b
        a = width_mm / 2
        b = height_mm / 2
        c = (a + b) / 2  # Estimate depth
        volume_ellipsoid = (4/3) * np.pi * a * b * c

        # Method 2: Rectangular prism (conservative estimate)
        depth_estimate = np.sqrt(width_mm * height_mm)  # Geometric mean
        volume_prism = width_mm * height_mm * depth_estimate

        # Method 3: Sphere approximation (for round tumors)
        diameter = (width_mm + height_mm) / 2
        radius = diameter / 2
        volume_sphere = (4/3) * np.pi * (radius ** 3)

        # Average volume (most reliable estimate)
        volume_avg = (volume_ellipsoid + volume_prism + volume_sphere) / 3

        # Calculate tumor characteristics
        aspect_ratio = width_mm / height_mm if height_mm > 0 else 0
        circularity = self._calculate_circularity(region)

        # Relative position in brain
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        relative_pos = {
            'x_ratio': center_x / img_shape[1],
            'y_ratio': center_y / img_shape[0],
            'quadrant': self._determine_quadrant(center_x, center_y, img_shape)
        }

        # Texture analysis
        texture = self._analyze_texture(region)

        return {
            'class': class_name,
            'confidence': confidence,
            'bbox': bbox,
            # 2D measurements
            'width_mm': width_mm,
            'height_mm': height_mm,
            'area_mm2': area_mm2,
            'diameter_avg_mm': (width_mm + height_mm) / 2,
            # 3D volume estimates
            'volume_ellipsoid_mm3': volume_ellipsoid,
            'volume_prism_mm3': volume_prism,
            'volume_sphere_mm3': volume_sphere,
            'volume_average_mm3': volume_avg,
            'volume_ml': volume_avg / 1000,  # Convert to milliliters
            # Shape characteristics
            'aspect_ratio': aspect_ratio,
            'circularity': circularity,
            'shape_regularity': self._classify_shape(aspect_ratio, circularity),
            # Position
            'position': relative_pos,
            # Texture
            'texture': texture
        }

    def _calculate_circularity(self, region):
        """Calculate circularity of tumor region (1.0 = perfect circle)"""

        gray = cv2.cvtColor(region, cv2.COLOR_RGB2GRAY)
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            area = cv2.contourArea(largest_contour)
            perimeter = cv2.arcLength(largest_contour, True)

            if perimeter > 0:
                circularity = 4 * np.pi * area / (perimeter ** 2)
                return min(circularity, 1.0)

        return 0.0

    def _classify_shape(self, aspect_ratio, circularity):
        """Classify tumor shape based on measurements"""

        if circularity > 0.8:
            return "Round/Spherical"
        elif 0.8 <= aspect_ratio <= 1.2:
            return "Oval/Elliptical"
        elif aspect_ratio > 1.5 or aspect_ratio < 0.67:
            return "Elongated/Irregular"
        else:
            return "Moderately Irregular"

    def _determine_quadrant(self, x, y, shape):
        """Determine which quadrant of the brain the tumor is in"""

        mid_x = shape[1] / 2
        mid_y = shape[0] / 2

        if x < mid_x and y < mid_y:
            return "Upper Left"
        elif x >= mid_x and y < mid_y:
            return "Upper Right"
        elif x < mid_x and y >= mid_y:
            return "Lower Left"
        else:
            return "Lower Right"

    def _analyze_texture(self, region):
        """Analyze texture characteristics of tumor"""

        if region.size == 0:
            return {'mean_intensity': 0, 'std_intensity': 0, 'contrast': 0}

        gray = cv2.cvtColor(region, cv2.COLOR_RGB2GRAY)

        return {
            'mean_intensity': np.mean(gray),
            'std_intensity': np.std(gray),
            'contrast': np.max(gray) - np.min(gray),
            'entropy': self._calculate_entropy(gray)
        }

    def _calculate_entropy(self, gray_image):
        """Calculate image entropy (texture complexity)"""

        histogram = cv2.calcHist([gray_image], [0], None, [256], [0, 256])
        histogram = histogram.ravel() / histogram.sum()

        # Remove zero entries
        histogram = histogram[histogram > 0]

        entropy = -np.sum(histogram * np.log2(histogram))
        return entropy

# ============================================
# 2. Multi-Image Volumetric Analysis
# ============================================

def analyze_multiple_images(model_path, image_folder, pixel_spacing=1.0, slice_thickness=5.0):
    """
    Analyze multiple images from a scan sequence

    Args:
        model_path: Path to YOLO model
        image_folder: Folder containing sequential scan images
        pixel_spacing: Physical size of pixel in mm
        slice_thickness: Distance between slices in mm
    """

    analyzer = BrainTumorVolumetricAnalyzer(model_path, pixel_spacing, slice_thickness)

    # Get all images
    image_paths = sorted(Path(image_folder).glob('*.jpg'))

    all_analyses = []
    total_volume = 0

    print(f"Analyzing {len(image_paths)} images...")

    for i, img_path in enumerate(image_paths):
        analyses, results, _ = analyzer.analyze_single_image(str(img_path))

        for analysis in analyses:
            analysis['slice_number'] = i
            analysis['image_name'] = img_path.name
            all_analyses.append(analysis)
            total_volume += analysis['volume_average_mm3']

    # Calculate aggregate statistics
    if all_analyses:
        df = pd.DataFrame(all_analyses)

        aggregate_stats = {
            'total_detections': len(all_analyses),
            'total_volume_mm3': total_volume,
            'total_volume_ml': total_volume / 1000,
            'avg_confidence': df['confidence'].mean(),
            'avg_diameter_mm': df['diameter_avg_mm'].mean(),
            'volume_range': [df['volume_average_mm3'].min(), df['volume_average_mm3'].max()],
            'tumor_classes': df['class'].value_counts().to_dict()
        }

        return df, aggregate_stats

    return None, None

# ============================================
# 3. Visualization Functions
# ============================================

def visualize_volumetric_analysis(analysis, img_rgb, results, output_path='volumetric_analysis.png'):
    """Create comprehensive volumetric visualization"""

    if not analysis:
        print("No detections to visualize")
        return

    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 4, hspace=0.4, wspace=0.4)

    # 1. Original image with detection
    ax1 = fig.add_subplot(gs[0, :2])
    pred_img = results[0].plot()
    ax1.imshow(cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB))
    ax1.set_title('Detected Brain Tumor', fontsize=16, fontweight='bold')
    ax1.axis('off')

    # 2. Volumetric measurements bar chart
    ax2 = fig.add_subplot(gs[0, 2:])
    volumes = [analysis[0]['volume_ellipsoid_mm3'],
               analysis[0]['volume_prism_mm3'],
               analysis[0]['volume_sphere_mm3'],
               analysis[0]['volume_average_mm3']]
    methods = ['Ellipsoid', 'Prism', 'Sphere', 'Average']
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

    bars = ax2.bar(methods, volumes, color=colors, edgecolor='black', linewidth=2)
    ax2.set_ylabel('Volume (mm³)', fontsize=12, fontweight='bold')
    ax2.set_title('Volume Estimation Methods', fontsize=14, fontweight='bold')
    ax2.grid(axis='y', alpha=0.3)

    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}',
                ha='center', va='bottom', fontweight='bold')

    # 3. 2D Measurements
    ax3 = fig.add_subplot(gs[1, :2])
    ax3.axis('off')

    measurements_text = f"""
    📏 2D MEASUREMENTS
    {'='*50}
    Width:              {analysis[0]['width_mm']:.2f} mm
    Height:             {analysis[0]['height_mm']:.2f} mm
    Area:               {analysis[0]['area_mm2']:.2f} mm²
    Average Diameter:   {analysis[0]['diameter_avg_mm']:.2f} mm

    📦 3D VOLUME ESTIMATES
    {'='*50}
    Ellipsoid Model:    {analysis[0]['volume_ellipsoid_mm3']:.2f} mm³
    Rectangular Model:  {analysis[0]['volume_prism_mm3']:.2f} mm³
    Spherical Model:    {analysis[0]['volume_sphere_mm3']:.2f} mm³

    ⭐ Average Volume:   {analysis[0]['volume_average_mm3']:.2f} mm³
    💧 Volume (mL):      {analysis[0]['volume_ml']:.4f} mL

    🔍 SHAPE ANALYSIS
    {'='*50}
    Aspect Ratio:       {analysis[0]['aspect_ratio']:.3f}
    Circularity:        {analysis[0]['circularity']:.3f}
    Shape Type:         {analysis[0]['shape_regularity']}

    📍 LOCATION
    {'='*50}
    Brain Quadrant:     {analysis[0]['position']['quadrant']}
    Relative X:         {analysis[0]['position']['x_ratio']:.3f}
    Relative Y:         {analysis[0]['position']['y_ratio']:.3f}
    """

    ax3.text(0.1, 0.95, measurements_text, transform=ax3.transAxes,
            fontsize=11, verticalalignment='top', family='monospace',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

    # 4. Shape characteristics radar chart
    ax4 = fig.add_subplot(gs[1, 2:], projection='polar')

    categories = ['Confidence', 'Circularity', 'Aspect\nRatio\n(norm)', 'Texture\nContrast\n(norm)']
    values = [
        analysis[0]['confidence'],
        analysis[0]['circularity'],
        min(analysis[0]['aspect_ratio'], 2.0) / 2.0,  # Normalize
        min(analysis[0]['texture']['contrast'] / 255.0, 1.0)
    ]

    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
    values += values[:1]
    angles += angles[:1]

    ax4.plot(angles, values, 'o-', linewidth=2, color='#FF6B6B')
    ax4.fill(angles, values, alpha=0.25, color='#FF6B6B')
    ax4.set_xticks(angles[:-1])
    ax4.set_xticklabels(categories)
    ax4.set_ylim(0, 1)
    ax4.set_title('Tumor Characteristics', fontsize=14, fontweight='bold', pad=20)
    ax4.grid(True)

    # 5. 3D Volume visualization (simplified representation)
    ax5 = fig.add_subplot(gs[2, :2], projection='3d')

    # Create ellipsoid representation
    a = analysis[0]['width_mm'] / 2
    b = analysis[0]['height_mm'] / 2
    c = (a + b) / 2

    u = np.linspace(0, 2 * np.pi, 50)
    v = np.linspace(0, np.pi, 50)
    x = a * np.outer(np.cos(u), np.sin(v))
    y = b * np.outer(np.sin(u), np.sin(v))
    z = c * np.outer(np.ones(np.size(u)), np.cos(v))

    ax5.plot_surface(x, y, z, alpha=0.7, cmap='Reds', edgecolor='none')
    ax5.set_xlabel('Width (mm)', fontweight='bold')
    ax5.set_ylabel('Height (mm)', fontweight='bold')
    ax5.set_zlabel('Depth (mm)', fontweight='bold')
    ax5.set_title('3D Tumor Model (Ellipsoid)', fontsize=14, fontweight='bold')

    # 6. Texture analysis
    ax6 = fig.add_subplot(gs[2, 2:])
    texture_metrics = ['Mean\nIntensity', 'Std Dev', 'Contrast', 'Entropy']
    texture_values = [
        analysis[0]['texture']['mean_intensity'] / 255,
        analysis[0]['texture']['std_intensity'] / 128,
        analysis[0]['texture']['contrast'] / 255,
        analysis[0]['texture']['entropy'] / 8
    ]

    colors_texture = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']
    bars = ax6.barh(texture_metrics, texture_values, color=colors_texture, edgecolor='black', linewidth=2)
    ax6.set_xlabel('Normalized Value', fontsize=12, fontweight='bold')
    ax6.set_title('Texture Characteristics', fontsize=14, fontweight='bold')
    ax6.set_xlim(0, 1)
    ax6.grid(axis='x', alpha=0.3)

    # Add value labels
    for i, (bar, val) in enumerate(zip(bars, texture_values)):
        ax6.text(val + 0.02, i, f'{val:.3f}', va='center', fontweight='bold')

    # Main title
    tumor_class = analysis[0]['class']
    confidence = analysis[0]['confidence'] * 100
    plt.suptitle(f'Brain Tumor Volumetric Analysis\n{tumor_class.upper()} - {confidence:.1f}% Confidence',
                fontsize=18, fontweight='bold', y=0.98)

    plt.savefig(output_path, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"✅ Volumetric analysis saved to: {output_path}")
    plt.show()

# ============================================
# 4. Generate Detailed Report
# ============================================

def generate_volumetric_report(analysis):
    """Generate detailed volumetric analysis report"""

    if not analysis:
        print("No detections to report")
        return

    a = analysis[0]

    print("\n" + "="*80)
    print("🧠 BRAIN TUMOR VOLUMETRIC ANALYSIS REPORT")
    print("="*80)

    print(f"\n📋 TUMOR CLASSIFICATION")
    print("-"*80)
    print(f"  Type:           {a['class'].upper()}")
    print(f"  Confidence:     {a['confidence']*100:.2f}%")
    print(f"  Detection Box:  ({a['bbox'][0]}, {a['bbox'][1]}) → ({a['bbox'][2]}, {a['bbox'][3]})")

    print(f"\n📏 2D MEASUREMENTS")
    print("-"*80)
    print(f"  Width:                {a['width_mm']:.2f} mm")
    print(f"  Height:               {a['height_mm']:.2f} mm")
    print(f"  Cross-sectional Area: {a['area_mm2']:.2f} mm²")
    print(f"  Average Diameter:     {a['diameter_avg_mm']:.2f} mm")

    print(f"\n📦 3D VOLUME ESTIMATES")
    print("-"*80)
    print(f"  Ellipsoid Model:      {a['volume_ellipsoid_mm3']:.2f} mm³")
    print(f"  Rectangular Model:    {a['volume_prism_mm3']:.2f} mm³")
    print(f"  Spherical Model:      {a['volume_sphere_mm3']:.2f} mm³")
    print(f"  ───────────────────────────────────────")
    print(f"  Average Estimate:     {a['volume_average_mm3']:.2f} mm³")
    print(f"  Volume in mL:         {a['volume_ml']:.4f} mL")
    print(f"  Volume in cm³:        {a['volume_average_mm3']/1000:.4f} cm³")

    print(f"\n🔍 SHAPE ANALYSIS")
    print("-"*80)
    print(f"  Aspect Ratio:         {a['aspect_ratio']:.3f}")
    print(f"  Circularity Index:    {a['circularity']:.3f} (1.0 = perfect circle)")
    print(f"  Shape Classification: {a['shape_regularity']}")

    print(f"\n📍 ANATOMICAL LOCATION")
    print("-"*80)
    print(f"  Brain Quadrant:       {a['position']['quadrant']}")
    print(f"  Horizontal Position:  {a['position']['x_ratio']*100:.1f}% from left")
    print(f"  Vertical Position:    {a['position']['y_ratio']*100:.1f}% from top")

    print(f"\n🎨 TEXTURE CHARACTERISTICS")
    print("-"*80)
    print(f"  Mean Intensity:       {a['texture']['mean_intensity']:.2f}")
    print(f"  Intensity Std Dev:    {a['texture']['std_intensity']:.2f}")
    print(f"  Contrast:             {a['texture']['contrast']:.2f}")
    print(f"  Entropy:              {a['texture']['entropy']:.2f}")

    print(f"\n💡 CLINICAL NOTES")
    print("-"*80)

    # Size classification
    if a['diameter_avg_mm'] < 10:
        size_class = "Small (<10mm)"
        note = "May require monitoring"
    elif a['diameter_avg_mm'] < 30:
        size_class = "Medium (10-30mm)"
        note = "Clinical evaluation recommended"
    else:
        size_class = "Large (>30mm)"
        note = "Immediate clinical attention advised"

    print(f"  Size Classification:  {size_class}")
    print(f"  Note:                 {note}")

    # Shape analysis
    if a['circularity'] > 0.8:
        shape_note = "Regular round shape - typical for benign tumors"
    elif a['circularity'] < 0.5:
        shape_note = "Irregular shape - may warrant further investigation"
    else:
        shape_note = "Moderate irregularity - standard clinical protocols apply"

    print(f"  Shape Assessment:     {shape_note}")

    print("\n" + "="*80)
    print("⚠️  DISCLAIMER: This analysis is for research purposes only.")
    print("    All clinical decisions should be made by qualified medical professionals.")
    print("="*80 + "\n")

# ============================================
# 5. Main Execution
# ============================================

# Configuration
MODEL_PATH = "/content/runs/detect/train/weights/best.pt"
IMAGE_PATH = "/content/Brain-Tumor-Detection-1/test/images/Te-me_0159_jpg.rf.7afad29587df72dcfba2e189a75656fa.jpg"

# Set physical parameters (adjust based on your MRI/CT scan specifications)
PIXEL_SPACING = 0.5  # mm per pixel (typical for brain MRI: 0.5-1.0 mm)
SLICE_THICKNESS = 5.0  # mm between slices (typical: 1-5 mm)

print("🔬 Initializing Brain Tumor Volumetric Analyzer...")
print(f"   Pixel Spacing: {PIXEL_SPACING} mm")
print(f"   Slice Thickness: {SLICE_THICKNESS} mm\n")

# Create analyzer
analyzer = BrainTumorVolumetricAnalyzer(MODEL_PATH, PIXEL_SPACING, SLICE_THICKNESS)

# Analyze image
print("🔍 Analyzing brain tumor...")
analyses, results, img_rgb = analyzer.analyze_single_image(IMAGE_PATH)

if analyses:
    print(f"✅ Detected {len(analyses)} tumor(s)\n")

    # Generate visualization
    print("📊 Creating volumetric visualization...")
    visualize_volumetric_analysis(analyses, img_rgb, results, 'brain_tumor_volumetric.png')

    # Display result
    from IPython.display import Image, display
    display(Image('brain_tumor_volumetric.png'))

    # Generate detailed report
    generate_volumetric_report(analyses)

    # Save results to JSON
    with open('volumetric_analysis.json', 'w') as f:
        json.dump(analyses, f, indent=2)
    print("💾 Analysis saved to: volumetric_analysis.json")

else:
    print("❌ No tumors detected in the image")